In [1]:
%%html

<style>
.jp-MarkdownOutput {
    font-size: 2.5em !important;
}
.jp-MarkdownOutput table {
    font-size: 1em !important;
}
.jp-OutputArea-output pre {
    font-size: 2em !important;
}
.cm-content {
    font-size: 2em !important;
}
.page-id-xx, html {
    scrollbar-width: none; /* FF */
}
::-webkit-scrollbar {
    width: 0px; /* Chrome & Edge */
}
.jp-Notebook.jp-mod-commandMode .jp-Cell.jp-mod-selected {
    background: none;
}
</style>

<img src="img/title.svg" style="width: 90%; border: 1px solid black; margin: 3em auto;">

## Outline

* Who wants high performance and why?
* Understanding why Python is slow
* Python escape hatches:
  * NumPy
  * Awkward Array
  * JIT-compilation
* Special topics:
  * JIT-compilation for GPUs
  * The Python garbage collector
  * The Python GIL (Global Interpreter Lock)
  * Memory-mapping
* No conclusions: we'll stop when we run out of time

## Who wants high performace and why?

* "I'm shepherding a computation that runs for months and 5% faster means 5% less electricity, operating costs, and $CO_2$."

* "I'm building an interactive app, and if each user-initiated action completes in less than human reaction time (100 ms), the app will feel snappy."

* "I'm analyzing a large dataset, and if the computation completes over a lunch break instead of overnight, I'll be able to run it more often to do more in-depth investigations."

* "I'm trying to do &lt;anything at all&gt;, and I can't because I run out of RAM, open file handles, TTYs, ..."

This is me:

* "I'm analyzing a large dataset, and if the computation completes over a lunch break instead of overnight, I'll be able to run it more often to do more in-depth investigations."
* "I'm trying to do &lt;anything at all&gt;, and I can't because I run out of RAM, open file handles, TTYs, ..."

<br><br><br>

Therefore, I usually only care about **speed** when it's an order of magnitude gain and **memory** when necessary.

<img src="img/clock-rate-timeline-1.svg" style="width: 90%; margin: 3em auto;">

<img src="img/clock-rate-timeline-2.svg" style="width: 90%; margin: 3em auto;">

<img src="img/clock-rate-timeline-3.svg" style="width: 90%; margin: 3em auto;">

<img src="img/clock-rate-timeline-4.svg" style="width: 90%; margin: 3em auto;">

There are now only three ways to speed up code:

1. parallel processing

2. fixing bloopers: what you thought it was doing is not what it's doing, or there's a faster method you just didn't know about

3. turning off dynamic features that you don't need

<img src="img/dynamic-features.svg" style="width: 60%; margin: 3em auto;">

"Dynamic feature":

* decisions are made in the loop that scales with dataset size

<br>

"that you don't need":

* information is known before starting that loop; it doesn't need to be re-derived

Dynamic features are often built into the language

| | dynamic allocation | reference count | garbage collector | runtime evaluation | virtual machine | type reflection | parallel scheduling |
|:--|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
| Fortran77 | | | | | | | |
| C | ✓ | | | | | | |
| C++ | ✓ | `shared_ptr<T>` | | | | vtable | stdlib |
| Rust | ✓ | `Rc<T>` | | | | vtable | ✓ |
| Swift | ✓ | ✓ | | | | vtable | ✓ |
| Julia | ✓ | | ✓ | ✓ | | ✓ | stdlib |
| Go | ✓ | | ✓ | | | vtable | ✓ |
| JVM | ✓ | | ✓ | | ✓ | ✓ | stdlib |
| Lua | ✓ | | ✓ | ✓ | ✓ | ✓ | |
| Python | ✓ | ✓ | ✓ | ✓ | ✓ | ✓ | stdlib |

<img src="img/benchmark-games-2023.svg" style="width: 80%; margin: 3em auto;">

## Understanding why Python is slow

How do dynamic features slow down a calculation?

<br>

In [2]:
import numpy as np

<br>

In [3]:
million_integers = np.random.normal(0, 10, 1_000_000).astype(np.int32)
million_integers

array([ -7,  -2,   2, ...,   7,   6, -15], shape=(1000000,), dtype=int32)

<br>

In [4]:
def addthem_python(data):
    out = 0
    for x in data:
        out += x
    return out

In [5]:
%%writefile addthem.c

int run(int* data) {
    int out = 0;
    for (int i = 0;  i < 1000000;  i++) {
        out += data[i];
    }
    return out;
}

Overwriting addthem.c


<br>

In [6]:
!cc -O0 -shared addthem.c -o libaddthem_attempt1.so

<br>

In [7]:
import ctypes

<br>

In [8]:
libaddthem = ctypes.cdll.LoadLibrary("./libaddthem_attempt1.so")
pointer_to_ints = ctypes.POINTER(ctypes.c_int)
libaddthem.run.argtypes = (pointer_to_ints,)
libaddthem.run.restype = ctypes.c_int

In [9]:
addthem_python(million_integers)

np.int32(7685)

<br>

In [10]:
libaddthem.run(million_integers.ctypes.data_as(pointer_to_ints))

7685

<br>

In [11]:
%%timeit

addthem_python(million_integers)

51.7 ms ± 1.44 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


<br>

In [12]:
%%timeit

libaddthem.run(million_integers.ctypes.data_as(pointer_to_ints))

853 μs ± 4.8 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


To see what's slowing Python down, let's look at a toy language with similar dynamic features

<br>

In [13]:
!c++ -std=c++11 -O3 baby-python.cpp -o baby-python

<br>

In [14]:
%%bash

./baby-python <<EOF
123
add(3, 5)
square_them = def(x) mul(x, x)
map(square_them, [1, 2, 3, 4, 5])
EOF

                     num = -123        add(x, x)   get(lst, i)   map(f, lst)
               oo    lst = [1, 2, 3]   mul(x, x)   len(lst)      reduce(f, lst)
. . . __/\_/\_/`'    f = def(x) single-expr   f = def(x, y) { ... ; last-expr }

>> 123
(1.397e-06 seconds)
>> 8
(3.352e-06 seconds)
>> <user-defined function>
(1.886e-06 seconds)
>> [1, 4, 9, 16, 25]
(5.168e-06 seconds)
>> 


In [15]:
million_integers.tofile("million_integers.int32")

<br>

In [16]:
%%bash

./baby-python data=million_integers.int32 <<EOF
reduce(add, data)
reduce(add, data)
reduce(add, data)
EOF

                     num = -123        add(x, x)   get(lst, i)   map(f, lst)
               oo    lst = [1, 2, 3]   mul(x, x)   len(lst)      reduce(f, lst)
. . . __/\_/\_/`'    f = def(x) single-expr   f = def(x, y) { ... ; last-expr }

>> 7685
(0.0895386 seconds)
>> 7685
(0.0885569 seconds)
>> 7685
(0.0883256 seconds)
>> 


What baby-python is doing:

* variables are in an `unordered_map<string, shared_ptr<Object>>`
* `Object` is an abstract class; C++ has to maintain a vtable to keep track of which type each concrete instance is (runtime polymorphism)
* instructions are represented by a data structure that must be traversed (abstract syntax tree, or AST)
* parsing and data-loading happen before the loop over data and are not slowing it down

<br>

In [17]:
from pygments import highlight
from pygments.lexers import CppLexer
from pygments.formatters import HtmlFormatter
from IPython.display import display, HTML

<br>

In [18]:
with open("baby-python.cpp") as file:
    display(HTML(highlight(file.read(), CppLexer(), HtmlFormatter())))

Similarly in Python:

* variables are in a `dict[str, object]`

<br>

In [19]:
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  "get_ipython().run_cell_magic('html', '', '\\n<style>\\n.jp-MarkdownOutput {\\n    font-size: 2.5em !important;\\n}\\n.jp-MarkdownOutput table {\\n    font-size: 1em !important;\\n}\\n.jp-OutputArea-output pre {\\n    font-size: 2em !important;\\n}\\n.cm-content {\\n    font-size: 2em !important;\\n}\\n.page-id-xx, html {\\n    scrollbar-width: none; /* FF */\\n}\\n::-webkit-scrollbar {\\n    width: 0px; /* Chrome & Edge */\\n}\\n.jp-Notebook.jp-mod-commandMode .jp-Cell.jp-mod-selected {\\n    background: none;\\n}\\n</style>\\n')",
  'import numpy as np',
  'million_integers = np.random.normal(0, 10, 1_000_000).astype(np.int32)\nmillion_integers',
  'def addthem_python(data):\n    out = 0\n    for x in data:\n 

Similarly in Python:

* all data share an overloaded struct type (implemented in C, rather than using C++ to generate vtables automatically)

```c
struct PyObject {
    Py_size_t ob_refcnt;    // reference count for garbage collection
    PyObject* ob_type;      // the object's type (also a PyObject)
                            // more fields that depend on type
};
```

In [20]:
class PyObject(ctypes.Structure):
    pass

PyObject._fields_ = [
    ("ob_refcnt", ctypes.c_size_t),
    ("ob_type", ctypes.POINTER(PyObject)),
]

<br>

In [21]:
some_string = "This is a nice string."

<br>

In [22]:
c_some_string = PyObject.from_address(id(some_string))

<br>

In [23]:
c_some_string.ob_refcnt

1

<br>

In [24]:
list_of_string = [some_string] * 100

<br>

In [25]:
ctypes.cast(c_some_string.ob_type, ctypes.c_void_p).value == id(str)

True

Similarly in Python:

* instructions are represented by a data structure that must be traversed (an array of bytecodes)

<br>

In [26]:
import dis

<br>

In [27]:
dis.dis(addthem_python)
# lineno|offset|opcode_name      |argument|description
# ------+------+-----------------+--------+------------

  1           0 RESUME                   0

  2           2 LOAD_CONST               1 (0)
              4 STORE_FAST               1 (out)

  3           6 LOAD_FAST                0 (data)
              8 GET_ITER
        >>   10 FOR_ITER                 7 (to 28)
             14 STORE_FAST               2 (x)

  4          16 LOAD_FAST                1 (out)
             18 LOAD_FAST                2 (x)
             20 BINARY_OP               13 (+=)
             24 STORE_FAST               1 (out)
             26 JUMP_BACKWARD            9 (to 10)

  3     >>   28 END_FOR

  5          30 LOAD_FAST                1 (out)
             32 RETURN_VALUE


<br>

None of the above instructions specify data types. The code that adds (`BINARY_OP 13`) has to determine that `x` is an integer, over and over, for a million values.

By contrast, the instructions generated by a compiler are passed directly to the CPU, and therefore have to be different instructions for different data types.

<br>

In [28]:
!cc -O0 -c addthem.c

<br>

In [29]:
!objdump -d addthem.o


addthem.o:     file format elf64-x86-64


Disassembly of section .text:

0000000000000000 <run>:
   0:	55                   	push   %rbp
   1:	48 89 e5             	mov    %rsp,%rbp
   4:	48 89 7d e8          	mov    %rdi,-0x18(%rbp)
   8:	c7 45 fc 00 00 00 00 	movl   $0x0,-0x4(%rbp)
   f:	c7 45 f8 00 00 00 00 	movl   $0x0,-0x8(%rbp)
  16:	eb 1d                	jmp    35 <run+0x35>
  18:	8b 45 f8             	mov    -0x8(%rbp),%eax
  1b:	48 98                	cltq
  1d:	48 8d 14 85 00 00 00 	lea    0x0(,%rax,4),%rdx
  24:	00 
  25:	48 8b 45 e8          	mov    -0x18(%rbp),%rax
  29:	48 01 d0             	add    %rdx,%rax
  2c:	8b 00                	mov    (%rax),%eax
  2e:	01 45 fc             	add    %eax,-0x4(%rbp)
  31:	83 45 f8 01          	addl   $0x1,-0x8(%rbp)
  35:	81 7d f8 3f 42 0f 00 	cmpl   $0xf423f,-0x8(%rbp)
  3c:	7e da                	jle    18 <run+0x18>
  3e:	8b 45 fc             	mov    -0x4(%rbp),%eax
  41:	5d                   	pop    %rbp
  42:	c3                  

## Python escape hatches: NumPy

<br>

"Making Python go fast" usually means "avoiding Python in hot loops."

NumPy provides an array type and suite of functions to do that.

<img src="img/python-list-layout.svg" style="width: 80%; margin: 3em auto;">

<img src="img/python-array-layout.svg" style="width: 80%; margin: 3em auto;">

In [30]:
%%timeit

addthem_python(million_integers)

50.4 ms ± 1.57 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


<br>

In [31]:
%%timeit

np.sum(million_integers)

201 μs ± 868 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


<br>

But you know about NumPy (and Pandas, and everything else that uses NumPy).

## Python escape hatches: Awkward Array

<br>

Like NumPy, but for irregularly shaped data structures.

<img src="img/awkward-motivation-venn-diagram.svg" style="width: 30%; margin: 1em auto;">

In [32]:
import awkward as ak

In [33]:
ragged = ak.Array([
    [
      [[1.84, 0.324]],
      [[-1.609, -0.713, 0.005], [0.953, -0.993, 0.011, 0.718]],
      [[0.459, -1.517, 1.545], [0.33, 0.292]],
      [[-0.376, -1.46, -0.206], [0.65, 1.278]],
      [[], [], [1.617]],
      []
    ],
    [
      [[-0.106, 0.611]],
      [[0.118, -1.788, 0.794, 0.658], [-0.105]]
    ],
    [
      [[-0.384], [0.697, -0.856]],
      [[0.778, 0.023, -1.455, -2.289], [-0.67], [1.153, -1.669, 0.305, 1.517, -0.292]]
    ],
    [
      [[0.205, -0.355], [-0.265], [1.042]],
      [[-0.004], [-1.167, -0.054, 0.726, 0.213]],
      [[1.741, -0.199, 0.827]]
    ]
])

In [34]:
print(ragged[3, 1, -1, 2])

0.726


<br>

In [35]:
print(ragged[3, 1:, -1, 1:3])

[[-0.054, 0.726], [-0.199, 0.827]]


<br>

In [36]:
print(ragged[[False, False, True, True], [0, -1, 0, -1], 0, -1])

[-0.384, 0.827]


<br>

In [37]:
print(ragged[ragged > 0])

[[[[1.84, 0.324]], [[0.005], [0.953, 0.011, 0.718]], ..., [[], ...], []], ...]


<br>

In [38]:
print(ak.sum(ragged))

2.8980000000000006


<img src="img/example-reducer-2d.svg" style="width: 40%; margin: 1em auto;">

In [39]:
regular = np.array([
    [  1,   2,   3,   4],
    [ 10,  20,  30,  40],
    [100, 200, 300, 400],
])

<br>

In [40]:
np.sum(regular, axis=0)

array([111, 222, 333, 444])

<br>

In [41]:
np.sum(regular, axis=1)

array([  10,  100, 1000])

<img src="img/example-reduction-sum.svg" style="width: 40%; margin: 1em auto;">

In [42]:
irregular = ak.Array([
    [   1,    2,    4],
    [                ],
    [None,    8      ],
    [  16            ],
])

<br>

In [43]:
print(ak.sum(irregular, axis=0))

[17, 10, 4]


<br>

In [44]:
print(ak.sum(irregular, axis=1))

[7, 0, 8, 16]


In [45]:
ragged.layout

<ListOffsetArray len='4'>
    <offsets><Index dtype='int64' len='5'>
        [ 0  6  8 10 13]
    </Index></offsets>
    <content><ListOffsetArray len='13'>
        <offsets><Index dtype='int64' len='14'>
            [ 0  1  3  5  7 10 10 11 13 15 18 21 23 24]
        </Index></offsets>
        <content><ListOffsetArray len='24'>
            <offsets><Index dtype='int64' len='25'>
                [ 0  2  5  9 12 14 17 19 19 19 20 22 26 27 28 30 34 35 40 42 43
                 44 45 49 52]
            </Index></offsets>
            <content><NumpyArray dtype='float64' len='52'>
                [ 1.84   0.324 -1.609 -0.713  0.005  0.953 -0.993  0.011  0.718
                  0.459 -1.517  1.545  0.33   0.292 -0.376 -1.46  -0.206  0.65
                 ...
                 -1.669  0.305  1.517 -0.292  0.205 -0.355 -0.265  1.042 -0.004
                 -1.167 -0.054  0.726  0.213  1.741 -0.199  0.827]
            </NumpyArray></content>
        </ListOffsetArray></content>
    </ListOffset

<br>

All operations are implemented as compiled functions on `<NumpyArray>` and `<Index>` arrays, which don't need to know a node's `<content>` type.

In [46]:
million_records = ak.from_parquet("data/million_records.parquet")
million_records

<Array [[{x: 2.46, y: [...]}, ..., {...}], ...] type='1000000 * var * {x: f...'>

In [47]:
%%timeit

np.square(million_records["y", ..., 1:])

142 ms ± 896 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


<br>

In [52]:
million_dicts = ak.to_list(million_records)

<br>

In [53]:
%%timeit -n1 -r1

output = []
for sublist in million_dicts:
    tmp1 = []
    for record in sublist:
        tmp2 = []
        for number in record["y"][1:]:
            tmp2.append(np.square(number))
        tmp1.append(tmp2)
    output.append(tmp1)

8.69 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<br>

Also, `million_records` uses 10× less memory than `million_dicts`.

## Python escape hatches: JIT-compilation

<br>

Despite being much faster than pure Python, array-computations are still a few times slower than they could be.

In [59]:
def quadratic_formula(a: np.array, b: np.array, c: np.array) -> np.array:
    return (-b + np.sqrt(b**2 - 4*a*c)) / (2*a)

<br>

runs whole arrays through each operation before moving on to the next, like this:

<br>

In [60]:
def pedantic_quadratic_formula(a: np.array, b: np.array, c: np.array) -> np.array:
    tmp1 = np.negative(b)            # -b
    tmp2 = np.square(b)              # b**2
    tmp3 = np.multiply(4, a)         # 4*a
    tmp4 = np.multiply(tmp3, c)      # tmp3*c
    del tmp3
    tmp5 = np.subtract(tmp2, tmp4)   # tmp2 - tmp4
    del tmp2, tmp4
    tmp6 = np.sqrt(tmp5)             # sqrt(tmp5)
    del tmp5
    tmp7 = np.add(tmp1, tmp6)        # tmp1 + tmp6
    del tmp1, tmp6
    tmp8 = np.multiply(2, a)         # 2*a
    return np.divide(tmp7, tmp8)     # tmp7 / tmp8

In [61]:
a = np.random.uniform(5, 10, 5_000_000)
b = np.random.uniform(10, 20, 5_000_000)
c = np.random.uniform(-0.1, 0.1, 5_000_000)

<br>

In [62]:
%%timeit

quadratic_formula(a, b, c)

65.1 ms ± 646 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


<br>

In [63]:
%%timeit

pedantic_quadratic_formula(a, b, c)

64.1 ms ± 630 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


But if we compile the formula, we can make it only one loop over the arrays.

<br>

In [64]:
%%writefile quadratic_formula_pybind11.cpp

#include <pybind11/pybind11.h>
#include <pybind11/numpy.h>
namespace py = pybind11;

void run(py::array_t<double, py::array::c_style | py::array::forcecast> a_numpy,
         py::array_t<double, py::array::c_style | py::array::forcecast> b_numpy,
         py::array_t<double, py::array::c_style | py::array::forcecast> c_numpy,
         py::array_t<double> output_numpy) {
    const double* a = a_numpy.data();
    const double* b = b_numpy.data();
    const double* c = c_numpy.data();
    double* output = output_numpy.mutable_data();
    for (int i = 0;  i < output_numpy.size();  i++) {
        output[i] = (-b[i] + sqrt(b[i]*b[i] - 4*a[i]*c[i])) / (2*a[i]);
    }
}

PYBIND11_MODULE(quadratic_formula_pybind11, m) {
    m.def("run", &run);
}

Writing quadratic_formula_pybind11.cpp


In [65]:
import os
import sys
from pybind11 import get_include

inc = "-I " + get_include()
plat = "-undefined dynamic_lookup" if "darwin" in sys.platform else "-fPIC"
pyinc = !python3-config --cflags

<br>

In [66]:
!c++ -std=c++11 quadratic_formula_pybind11.cpp -shared {inc} {pyinc.s} -o quadratic_formula_pybind11.so {plat}

<br>

In [67]:
import quadratic_formula_pybind11

<br>

In [70]:
output = np.zeros(len(a), dtype=np.float64)
quadratic_formula_pybind11.run(a, b, c, output)
output

array([-0.00608737, -0.00675183, -0.0018753 , ...,  0.0023538 ,
        0.00079179, -0.00505813], shape=(5000000,))

In [69]:
%%timeit

quadratic_formula(a, b, c)

64.7 ms ± 349 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


<br>

In [71]:
%%timeit

output = np.zeros(len(a), dtype=np.float64)
quadratic_formula_pybind11.run(a, b, c, output)

18.9 ms ± 109 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


<br>

Accessing memory is slower than most mathematical operations, so doing a lot of math in one pass is better than doing a little math in many passes.

Many, many libraries try to make it easy to integrate compilation into Python workflows.

<img src="img/history-of-bindings-2.svg" style="width: 80%; margin: 1em auto;">

## Special topics: JIT-compilation for GPUs

## Special topics: the Python garbage collector

## Special topics: the Python GIL (Global Interpreter Lock)

## Special topics: memory mapping